<a href="https://colab.research.google.com/github/habebaRabie/chatGPT-sentiment-analysis/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import numpy as np
import random
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten, GlobalAveragePooling1D, Dense, Input, GlobalMaxPooling1D, Concatenate, Dropout
from keras.models import Sequential, Model
from keras.optimizers import RMSprop, Adam
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/glove/file.csv')


In [ ]:
data.rename( columns={'Unnamed: 0':'id'}, inplace=True )

In [ ]:
my_dict = {data['id'][i]: [data['tweets'][i], data['labels'][i]] for i in range(len(data['id']))}

In [ ]:
#mapping the labels of the tweets to 0, 1, 2 instead of bad, neutral, good
mapping = {'bad': 0, 'neutral': 1, 'good': 2}
my_dict = {key: [value[0], mapping[value[1]]] for key, value in my_dict.items()}

In [ ]:
# texts = []

stop_words = set(stopwords.words('english'))
def preprocess(value):
# remove the URLs from the tweet body
  url_regex = re.compile(r'https?://\S+')
  value = url_regex.sub('', value)

  # remove the english symbols and \n from the tweet body
  replaces = value.replace("/", "")
  replaces = replaces.replace("\\n", " ")
  value = re.sub('[\W]', ' ',replaces)

  # remove the stop words from the tweet body
  words = value.split()
  filtered_words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
  value = ' '.join(filtered_words)
  return value

In [ ]:
texts = []

for key in my_dict.keys():
  my_dict[key][0] = preprocess(my_dict[key][0])
  texts.append(my_dict[key][0])

print(my_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
lemmatizer = WordNetLemmatizer()

lemmated_tokens = []

for key in texts:
  lemmated_string = ""
  words = key.split()
  for word_2 in words:
    lemmated_string = lemmated_string + lemmatizer.lemmatize(word_2)+ " "
  lemmated_tokens.append(lemmated_string)
print(lemmated_tokens[0])

chatgpt optimizing language model dialogue openai 


In [ ]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 40000
def tokenize(tokens):
    tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
    tokenizer.fit_on_texts(tokens)
    sequences = tokenizer.texts_to_sequences(tokens)
    word_index = tokenizer.word_index # the dictionary
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    return word_index,data

In [ ]:
word_index ,data = tokenize(lemmated_tokens)
print(word_index)

{'chatgpt': 1, 'ai': 2, 'openai': 3, 'like': 4, 'google': 5, 'write': 6, 'new': 7, 'asked': 8, 'using': 9, 'use': 10, 'answer': 11, 'question': 12, 'code': 13, 'time': 14, 'make': 15, 'get': 16, 'chatbot': 17, 'know': 18, 'good': 19, 'one': 20, 'thing': 21, 'think': 22, 'human': 23, 'people': 24, 'see': 25, 'via': 26, 'search': 27, 'amp': 28, 'way': 29, 'could': 30, 'work': 31, 'would': 32, 'tool': 33, 'need': 34, 'day': 35, 'future': 36, 'model': 37, 'writing': 38, 'year': 39, 'world': 40, 'going': 41, 'gpt': 42, 'bot': 43, 'ask': 44, 'prompt': 45, 'language': 46, 'even': 47, 'really': 48, 'better': 49, 'job': 50, 'technology': 51, 'say': 52, 'used': 53, 'help': 54, 'first': 55, 'chat': 56, 'tech': 57, 'story': 58, 'content': 59, 'u': 60, 'take': 61, 'artificialintelligence': 62, 'much': 63, 'twitter': 64, 'intelligence': 65, 'text': 66, 'still': 67, 'give': 68, 'got': 69, 'also': 70, 'response': 71, 'essay': 72, 'well': 73, 'create': 74, 'article': 75, 'right': 76, 'great': 77, 'many

In [ ]:
VALIDATION_SPLIT = 0.2
y = [my_dict[key][1] for key in my_dict]

temp =[]
for i in y:
  if i == 0:
    temp_1=[1,0,0]
  elif i==1:
    temp_1 = [0,1,0]
  else:
      temp_1=[0,0,1]
  temp.append(temp_1)

X_train, X_test, y_train, y_test = train_test_split(data, temp, test_size=0.2, random_state=42)
X_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train_array = np.array(X_train)
X_test_array = np.array(X_test)
y_train_array = np.array(y_train)
y_test_array = np.array(y_test)
x_val_array = np.array(x_val)
y_val_array = np.array(y_val)


In [ ]:
EMBEDDING_DIM = 300
embeddings_index = {}
with open('/content/drive/MyDrive/glove/Glove/glove.6B.300d.txt', 'r') as f:
  for line in f:
    values = line.split(sep=' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))#+1 to include the zerors vector for non-existing words
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
    embedding_matrix[i] = embedding_vector
print ('Shape of Embedding Matrix: ',embedding_matrix.shape)

Shape of Embedding Matrix:  (83260, 300)


In [ ]:
embedding_layer = Embedding(len(word_index) + 1, #vocab size
    EMBEDDING_DIM, #embedding vector size
    weights=[embedding_matrix], #weights matrix
    input_length=MAX_SEQUENCE_LENGTH, #padded sequence length
    trainable=False)

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH, ), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

filter_sizes = [3, 3, 4]

conv_layers = []
for filter_size in filter_sizes:
    conv_layer = Conv1D(128, kernel_size=filter_size, activation='relu')(embedded_sequences)
    pool_layer = GlobalMaxPooling1D()(conv_layer)
    conv_layers.append(pool_layer)

merged_layer = Concatenate()(conv_layers)
flatten_layer = Flatten()(merged_layer)
dropout_layer = Dropout(0.5)(flatten_layer)

dense_layer1 = Dense(128, activation='relu')(dropout_layer)
output_layer = Dense(3, activation='softmax')(dense_layer1)
# output_layer = Dense(3, activation='softmax')(merged_layer)
model = Model(inputs=sequence_input, outputs=output_layer)

In [ ]:
model = Model(sequence_input, output_layer)
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])


In [ ]:
model.fit(X_train_array, y_train_array, validation_data=(x_val_array, y_val_array),epochs=20, batch_size=128)

Epoch 1/20
1097/1097 [==============================] - 113s 92ms/step - loss: 0.6910 - acc: 0.6980 - val_loss: 0.5467 - val_acc: 0.7719
Epoch 2/20
1097/1097 [==============================] - 106s 96ms/step - loss: 0.5299 - acc: 0.7783 - val_loss: 0.4764 - val_acc: 0.8082
Epoch 3/20
1097/1097 [==============================] - 106s 97ms/step - loss: 0.4759 - acc: 0.8037 - val_loss: 0.4513 - val_acc: 0.8235
Epoch 4/20
1097/1097 [==============================] - 106s 96ms/step - loss: 0.4399 - acc: 0.8217 - val_loss: 0.4488 - val_acc: 0.8249
Epoch 5/20
1097/1097 [==============================] - 105s 96ms/step - loss: 0.4143 - acc: 0.8331 - val_loss: 0.4247 - val_acc: 0.8317
Epoch 6/20
1097/1097 [==============================] - 105s 96ms/step - loss: 0.3919 - acc: 0.8429 - val_loss: 0.4308 - val_acc: 0.8286
Epoch 7/20
1097/1097 [==============================] - 105s 96ms/step - loss: 0.3738 - acc: 0.8520 - val_loss: 0.4135 - val_acc: 0.8371
Epoch 8/20
1097/1097 [===================

In [ ]:
print('Acuracy on testing set:')
model.evaluate(X_test_array, y_test_array)

Acuracy on testing set:
1371/1371 [==============================] - 14s 10ms/step - loss: 0.4319 - acc: 0.8425


[0.43190786242485046, 0.8425180912017822]

In [ ]:
model.predict(X_test_array)

1371/1371 [==============================] - 12s 9ms/step


array([[9.98116612e-01, 1.43062242e-03, 4.52778797e-04],
       [9.94701505e-01, 2.18552654e-03, 3.11296759e-03],
       [9.99509931e-01, 3.71930568e-04, 1.18085933e-04],
       ...,
       [7.71004632e-02, 3.09468627e-01, 6.13430917e-01],
       [7.47859716e-01, 2.17508331e-01, 3.46319340e-02],
       [1.00248762e-04, 1.18561566e-01, 8.81338239e-01]], dtype=float32)

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))#+1 to include the zerors vector for non-existing words
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
    embedding_matrix[i] = embedding_vector
print ('Shape of Embedding Matrix: ',embedding_matrix.shape)

Shape of Embedding Matrix:  (83260, 300)


In [ ]:
embedding_layer = Embedding(len(word_index) + 1, #vocab size
    EMBEDDING_DIM, #embedding vector size
    weights=[embedding_matrix], #weights matrix
    input_length=MAX_SEQUENCE_LENGTH, #padded sequence length
    trainable=False)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH, ), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)


# Define the LSTM layer
lstm_layer = LSTM(128)(embedded_sequences)

# Add a dropout layer to reduce overfitting
dropout_layer = Dropout(0.5)(lstm_layer)

# Add a dense output layer with softmax activation for multiclass classification
output_layer = Dense(3, activation='softmax')(dropout_layer)

# Define the model with the input and output layers
LSTM_model = Model(inputs=sequence_input, outputs=output_layer)


In [ ]:
LSTM_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
LSTM_model.fit(X_train_array, y_train_array, validation_data=(x_val_array, y_val_array),epochs=20, batch_size=128)

Epoch 1/20
1097/1097 [==============================] - 100s 89ms/step - loss: 0.3928 - accuracy: 0.7250 - val_loss: 0.3157 - val_accuracy: 0.7946
Epoch 2/20
1097/1097 [==============================] - 97s 88ms/step - loss: 0.2929 - accuracy: 0.8143 - val_loss: 0.2674 - val_accuracy: 0.8327
Epoch 3/20
1097/1097 [==============================] - 99s 90ms/step - loss: 0.2546 - accuracy: 0.8449 - val_loss: 0.2429 - val_accuracy: 0.8512
Epoch 4/20
1097/1097 [==============================] - 99s 90ms/step - loss: 0.2245 - accuracy: 0.8664 - val_loss: 0.2373 - val_accuracy: 0.8570
Epoch 5/20
1097/1097 [==============================] - 98s 89ms/step - loss: 0.2058 - accuracy: 0.8798 - val_loss: 0.2309 - val_accuracy: 0.8647
Epoch 6/20
1097/1097 [==============================] - 98s 89ms/step - loss: 0.1911 - accuracy: 0.8908 - val_loss: 0.2235 - val_accuracy: 0.8697
Epoch 7/20
1097/1097 [==============================] - 95s 86ms/step - loss: 0.1773 - accuracy: 0.8998 - val_loss: 0.2262 

In [ ]:
print('Acuracy on testing set:')
LSTM_model.evaluate(X_test_array, y_test_array)

Acuracy on testing set:
1371/1371 [==============================] - 20s 15ms/step - loss: 0.3129 - accuracy: 0.8708


[0.31289801001548767, 0.8707677125930786]

In [ ]:
LSTM_model.predict(X_test_array)

1371/1371 [==============================] - 19s 13ms/step


array([[9.9994504e-01, 3.8836592e-05, 1.6040964e-05],
       [9.9999380e-01, 1.3294692e-06, 4.8314655e-06],
       [8.8767654e-01, 8.4595144e-02, 2.7728360e-02],
       ...,
       [3.2999297e-03, 9.9426752e-01, 2.4326192e-03],
       [9.9011046e-01, 7.1450616e-03, 2.7444572e-03],
       [1.1075454e-05, 1.6919936e-05, 9.9997199e-01]], dtype=float32)

In [ ]:
tweet = input("Enter your tweet: ")
print(tweet)

# {'bad': 0, 'neutral': 1, 'good': 2}
# tweet = "I had been seeing lots of tweets about ChatGPT so I signed up for it last night.  Turn out it works decently well for generating stories to read to your kids based on what they want to hear a story about."
tweet = preprocess(tweet)
# print(tweet)

lemmatize_token = []
words = tweet.split()
lemmatize_string = ""
for word_2 in words:
  lemmatize_string = lemmatize_string + lemmatizer.lemmatize(word_2)+" "
tweet = lemmatize_string
lemmatize_token.append(tweet)
# print(tweet)

tweet_word_index, tweet_data=tokenize(lemmatize_token)

# print(tweet_word_index)
# print(tweet_data)

label_vec = model.predict(tweet_data[0].reshape(1,-1))

actual_labels = {'negative': 0, 'neutral': 1, 'positive': 2}
probs = np.array(label_vec)
index = np.argmax(probs)
output_label = list(actual_labels.keys())[list(actual_labels.values()).index(index)]

print(output_label)

# print(label_vec)

Enter your tweet: One of my Christmas wishes this year: ChatGPT, please convert all the Terraform HCL templates you can find on the internet to Terraform CDK scripts, so that new projects won't ever have to deal with HCL syntax in this lifetime.
One of my Christmas wishes this year: ChatGPT, please convert all the Terraform HCL templates you can find on the internet to Terraform CDK scripts, so that new projects won't ever have to deal with HCL syntax in this lifetime.
1/1 [==============================] - 0s 18ms/step
positive
